In [1]:
# !pip3 install ipywidgets

In [2]:
import kagglehub
path = kagglehub.dataset_download("nih-chest-xrays/data")
print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/data


In [7]:
import numpy as np
import pandas as pd
import os
import requests
import json

In [8]:
!ls '/kaggle/input/data'

ARXIV_V5_CHESTXRAY.pdf	README_CHESTXRAY.pdf  images_005  images_010
BBox_List_2017.csv	images_001	      images_006  images_011
Data_Entry_2017.csv	images_002	      images_007  images_012
FAQ_CHESTXRAY.pdf	images_003	      images_008  test_list.txt
LOG_CHESTXRAY.pdf	images_004	      images_009  train_val_list.txt


In [9]:
df = pd.read_csv("/kaggle/input/data/Data_Entry_2017.csv")

In [10]:
df.fillna("", inplace=True)

/tmp/ipykernel_30/2065217620.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("", inplace=True)


In [11]:
df.describe()

,Follow-up #,Patient ID,Patient Age,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y]
count,112120.000000,112120.000000,112120.000000,112120.000000,112120.000000,112120.000000,112120.000000
mean,8.573751,14346.381743,46.901463,2646.078844,2486.438842,0.155649,0.155649
std,15.406320,8403.876972,16.839923,341.246429,401.268227,0.016174,0.016174
min,0.000000,1.000000,1.000000,1143.000000,966.000000,0.115000,0.115000
25%,0.000000,7310.750000,35.000000,2500.000000,2048.000000,0.143000,0.143000
50%,3.000000,13993.000000,49.000000,2518.000000,2544.000000,0.143000,0.143000
75%,10.000000,20673.000000,59.000000,2992.000000,2991.000000,0.168000,0.168000
max,183.000000,30805.000000,414.000000,3827.000000,4715.000000,0.198800,0.198800


In [12]:
df['Image Index'].iloc[26988]

'00007034_065.png'

In [13]:
df['Finding Labels']

0                   Cardiomegaly
1         Cardiomegaly|Emphysema
2          Cardiomegaly|Effusion
3                     No Finding
4                         Hernia
                   ...          
112115            Mass|Pneumonia
112116                No Finding
112117                No Finding
112118                No Finding
112119                No Finding
Name: Finding Labels, Length: 112120, dtype: object

In [16]:
new_df = df.reset_index(drop=True)

In [17]:
new_df.isnull().sum()

Image Index                    0
Finding Labels                 0
Follow-up #                    0
Patient ID                     0
Patient Age                    0
Patient Gender                 0
View Position                  0
OriginalImage[Width            0
Height]                        0
OriginalImagePixelSpacing[x    0
y]                             0
Unnamed: 11                    0
dtype: int64

In [18]:
new_df.shape

(112120, 12)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import random_split
from torch.utils.data import Dataset
from PIL import Image
import requests
import json

In [22]:
class CustomImageDataset(Dataset):
    def __init__(self, annotations, directories, label_list, transform=None):
        self.annotations = annotations
        self.directories = directories
        self.transform = transform
        self.label_list = label_list
        # Create a label map to assign each label an index
        self.label_map = {label: idx for idx, label in enumerate(self.label_list)}

    def find_image_path(self, image_name):
        for directory in self.directories:
            img_path = os.path.join(directory, image_name)
            if os.path.exists(img_path):
                return img_path
        raise FileNotFoundError(f"Image {image_name} not found in any of the provided directories.")
    
    def _encode_labels(self, labels_str):
        """
        Convert the labels in the format "label1|label2|..." to a multi-hot encoded vector.
        """
        labels = labels_str.split('|')
        label_vector = torch.zeros(len(self.label_list), dtype=torch.float32)
        for label in labels:
            if label in self.label_map:
                label_vector[self.label_map[label]] = 1.0 
        return label_vector

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        image_name = self.annotations['Image Index'].iloc[idx]
        img_path = self.find_image_path(image_name)
        image = Image.open(img_path).convert("RGB")
        
        # Extract and encode the labels
        label_str = self.annotations['Finding Labels'].iloc[idx]
        label_vector = self._encode_labels(label_str)

        if self.transform:
            image = self.transform(image)
        
        return image, label_vector

In [23]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),      
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [24]:
csv_file = '/kaggle/input/data/Data_Entry_2017.csv'
root_dir = '/kaggle/input/data/'
directories = [
    '/kaggle/input/data/images_001/images/',
    '/kaggle/input/data/images_002/images/',
    '/kaggle/input/data/images_003/images/',
    '/kaggle/input/data/images_004/images/',
    '/kaggle/input/data/images_005/images/',
    '/kaggle/input/data/images_006/images/',
    '/kaggle/input/data/images_007/images/',
    '/kaggle/input/data/images_008/images/',
    '/kaggle/input/data/images_009/images/',
    '/kaggle/input/data/images_010/images/',
    '/kaggle/input/data/images_011/images/',
    '/kaggle/input/data/images_012/images/'
]
label_list = ['Cardiomegaly', 'No Finding', 'Hernia', 'Infiltration', 'Nodule',
                  'Emphysema', 'Effusion', 'Atelectasis', 'Pleural_Thickening',
                  'Pneumothorax', 'Mass', 'Fibrosis', 'Consolidation', 'Edema', 'Pneumonia']

dataset = CustomImageDataset(
    annotations=new_df, directories=directories, label_list=label_list, transform=transform
)

In [25]:
dataset[0][0].shape

torch.Size([3, 224, 224])

In [26]:
dataset[0][1]

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [27]:
dataset_size = len(dataset)
train_size = int(0.7 * dataset_size)
val_size = int(0.15 * dataset_size)
test_size = dataset_size - train_size - val_size

In [28]:
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

In [29]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

In [30]:
next(iter(train_loader))[0].shape, next(iter(train_loader))[1].shape

(torch.Size([64, 3, 224, 224]), torch.Size([64, 15]))

In [32]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 15)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 28 * 28)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.fc4(x)
        return x


In [33]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [34]:
model = CNNModel().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [43]:
n_epochs = 5
threshold = 0.5

In [ ]:
for epoch in tqdm(range(n_epochs)):
    # Training Phase
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images) 
        loss = criterion(outputs, labels.float())
        
        # Backward and optimize
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        predicted = torch.sigmoid(outputs)
        predicted = (predicted > threshold).float()
        total_train += labels.size(0) * labels.size(1)
        correct_train += (predicted == labels).sum().item()

    train_accuracy = 100 * correct_train / total_train
    print(f'Epoch [{epoch+1}/{n_epochs}], Train Loss: {running_loss/len(train_loader):.4f}, Train Accuracy: {train_accuracy:.2f}%')
    
    # Validation Phase
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0
    
    with torch.no_grad():
        for images, labels in tqdm(val_loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels.float())
            
            val_loss += loss.item()
            
            predicted = torch.sigmoid(outputs)
            predicted = (predicted > threshold).float()
            total_val += labels.size(0) * labels.size(1)  # Total number of labels
            correct_val += (predicted == labels).sum().item()

    val_accuracy = 100 * correct_val / total_val
    print(f'Epoch [{epoch+1}/{n_epochs}], Validation Loss: {val_loss/len(val_loader):.4f}, Validation Accuracy: {val_accuracy:.2f}%')


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/1227 [00:00<?, ?it/s]

Epoch [1/5], Train Loss: 0.2057, Train Accuracy: 92.85%
ok


  0%|          | 0/263 [00:00<?, ?it/s]

Epoch [1/5], Validation Loss: 0.2031, Validation Accuracy: 92.85%
ok


  0%|          | 0/1227 [00:00<?, ?it/s]

In [2]:
# predicted

In [36]:
# Test Phase (after training)
model.eval()
correct_test = 0
total_test = 0

with torch.no_grad():
    for images, labels in tqdm(test_loader):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        
        predicted = torch.sigmoid(outputs)
        predicted = (predicted > threshold).float()
        total_test += labels.size(0) * labels.size(1)  # Total number of labels
        correct_test += (predicted == labels).sum().item()

test_accuracy = 100 * correct_test / total_test
print(f'Test Accuracy: {test_accuracy:.2f}%')
send_alert(f'Test Accuracy: {test_accuracy:.2f}%')

print('Training and evaluation complete!')

  0%|          | 0/263 [00:00<?, ?it/s]

Test Accuracy: 92.71%
ok
Training and evaluation complete!


In [38]:
torch.save(model.state_dict(), 'chest_xray_cnn_model.pth')